# Scraping NYT Articles and using NLP methods 
This is to identify and learn about potential EdTech donors!  

In [14]:
# might need to use Beautiful Soup and Pickle for this work too.
import requests
import os
from pprint import pprint

#my usual suspects
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
%matplotlib inline

In [60]:
nyt_apikey = open('../nyt_apikey.txt','r')

In [61]:
file = nyt_apikey.read().splitlines()

In [62]:
myapikey = file[0]

In [63]:
apikey = os.getenv('NYTIMES_APIKEY', myapikey)

## Look up for current top stories:
https://developer.nytimes.com/docs/top-stories-product/1/overview

In [64]:
section = "education"
query_url = f"https://api.nytimes.com/svc/topstories/v2/{section}.json?api-key={apikey}"

r = requests.get(query_url)
#pprint(r.json())

## Article Search Endpoint
https://developer.nytimes.com/docs/articlesearch-product/1/routes/articlesearch.json/get

In [53]:
query = "education"
begin_date = "20210925" # YYYYMMDD
#filter_query = "\"body:(\"space\") AND glocations:(\"BOSTON\")\""
#filter_query = "\"news_desk:(\"education\") AND body:(\"fund\") AND glocations:(\"NEW YORK CITY\")\""
filter_query = "\"news_desk: (\"education\") AND body:(\"fundraising\") \""
page = "0" # <0-100>
sort = "relevance" # newest, oldest

query_url = f"https://api.nytimes.com/svc/search/v2/articlesearch.json?" \
f"q={query}" \
f"&api-key={apikey}" \
f"&begin_date={begin_date}" \
f"&fq={filter_query}" \
f"&page={page}" \
f"&sort={sort}" \

r = requests.get(query_url)

jsonfile = r.json()
#pprint(jsonfile)

In [54]:
jsonfile.keys()

dict_keys(['status', 'copyright', 'response'])

In [55]:
len(jsonfile['response']['docs'])

10

In [69]:
articles = jsonfile['response']['docs']
articles[0].keys()

dict_keys(['abstract', 'web_url', 'snippet', 'lead_paragraph', 'source', 'multimedia', 'headline', 'keywords', 'pub_date', 'document_type', 'news_desk', 'section_name', 'byline', 'type_of_material', '_id', 'word_count', 'uri'])

In [100]:
data = {'abstract': [],
        'web_url': [],
        'lead_paragraph': [],
        'headline': [],  
        'date': [], 
        'doc_type': [],
        'material_type': [],
        'keywords': []}

In [94]:
import dateutil
from dateutil.relativedelta import relativedelta

In [96]:
# data = {'headline': [],  
#     'date': [], 
#     'doc_type': [],
#     'material_type': [],
#     'section': [],
#     'keywords': []}

# articles = jsonfile['response']['docs'] 
# for article in articles: # For each article, make sure it falls within our date range
#     date = dateutil.parser.parse(artile['pub_date']).date()
#     data['date'].append(date)
#     data['headline'].append(article['headline']['main']) 
#     if 'section' in article:
#         data['section'].append(article['section_name'])
#     else:
#         data['section'].append(None)
#     data['doc_type'].append(article['document_type'])
#     if 'type_of_material' in article: 
#         data['material_type'].append(article['type_of_material'])
#     else:
#         data['material_type'].append(None)
#     keywords = [keyword['value'] for keyword in article['keywords'] if keyword['name'] == 'subject']
#     data['keywords'].append(keywords)


In [101]:
for article in articles:
    data['abstract'].append(article['abstract'])
    data['web_url'].append(article['web_url'])
    data['lead_paragraph'].append(article['lead_paragraph'])
    data['headline'].append(article['headline']['main'])
    data['date'].append(article['pub_date'])
    data['doc_type'].append(article['document_type'])
    data['material_type'].append(article['type_of_material'])
    
    keywords = [keyword['value'] for keyword in article['keywords'] if keyword['name'] == 'subject']
    data['keywords'].append(keywords)

In [102]:
df = pd.DataFrame(data) 

In [103]:
df

,abstract,web_url,lead_paragraph,headline,date,doc_type,material_type,keywords
0,Following a collection of vignettes from schoo...,https://www.nytimes.com/2021/09/24/education/s...,"Last week, we published a special feature on s...",‘I’m So Tired’: Readers Respond to Schools Reo...,2021-09-24T09:00:25+0000,article,News,"[Education (K-12), Coronavirus Risks and Safet..."
1,Some districts have established robust virus t...,https://www.nytimes.com/2021/09/25/health/scho...,"SAN ANTONIO — One recent Thursday, Ciara Brown...","‘I Need an Army’: Across America, Schools Cram...",2021-09-25T13:00:13+0000,article,News,"[your-feed-science, Tests (Medical), Education..."
2,Pedro Martinez is leaving San Antonio to becom...,https://www.nytimes.com/2021/09/24/us/pedro-ma...,"For months, Pedro Martinez has been locked in ...",An Exit Interview With a School Superintendent...,2021-09-24T21:53:34+0000,article,News,"[Education (K-12), Teachers and School Employe..."
3,"In Tennessee, where the governor allows famili...",https://www.nytimes.com/2021/09/23/us/politics...,"FRANKLIN, Tenn. — Five years ago, Kim Hart’s s...","For Parents ​of Disabled Children, School Mask...",2021-09-23T07:00:16+0000,article,News,"[Special Education, Intellectual Disabilities,..."
4,The Biden administration is trying to address ...,https://www.nytimes.com/2021/09/21/upshot/chil...,"At a Y.M.C.A. in San Antonio, 200 children are...",‘Can’t Compete’: Why Hiring for Child Care Is ...,2021-09-21T09:00:19+0000,article,News,"[Child Care, Shortages, Children and Childhood..."
5,The rulings were the second and third the last...,https://www.nytimes.com/2021/09/25/us/tennesse...,Two federal judges in Tennessee have dealt blo...,Two more federal judges rule against the Tenne...,2021-09-25T17:24:59+0000,article,News,"[Special Education, Masks, Coronavirus (2019-n..."
6,Thousands of Department of Education employees...,https://www.nytimes.com/2021/09/24/nyregion/ny...,The nation’s largest school system is preparin...,N.Y.C. schools prepare for staffing shortages ...,2021-09-24T17:10:06+0000,article,News,"[Education (K-12), Teachers and School Employe..."
7,"The mandate, which affects well over 150,000 p...",https://www.nytimes.com/2021/09/25/nyregion/ny...,Mayor Bill de Blasio’s administration has been...,A federal appeals court delays New York City’s...,2021-09-25T14:04:40+0000,article,News,"[Coronavirus (2019-nCoV), Teachers and School ..."
8,Our focus on the problems with inner-city educ...,https://www.nytimes.com/2021/09/23/opinion/sub...,The Sweet Home Central School District serves ...,Have We Failed Suburban Schools?,2021-09-23T19:19:08+0000,article,Op-Ed,"[internal-sub-only, Education (K-12), Suburbs,..."
9,Some parents are hesitant to vaccinate 5-11-ye...,https://www.nytimes.com/2021/09/22/us/vaccines...,"Today, we’re covering a coming fight over vacc...",Vaccines for Young Kids?,2021-09-22T17:23:01+0000,article,News,"[Tests (Medical), Education (K-12), Shutdowns ..."


In [8]:
jsonfile['response']['docs'][0]['web_url']

'https://www.nytimes.com/2021/09/24/education/schools-reopening-comments.html'

In [10]:
def get_articles(file):
    
    article_results = []
    
    for i in range(len(file)):
        article_dict = {}
        
        article_dict['web_url'] = file[i]['web_url']
        article_dict['abstract'] = file[i]['abstract']
        
        article_results.append(article_dict)
        
    return article_results

In [11]:
list_articles = get_articles(jsonfile['response']['docs'])

In [12]:
len(list_articles)

10

In [13]:
list_articles[0]

{'web_url': 'https://www.nytimes.com/2021/09/24/education/schools-reopening-comments.html',
 'abstract': 'Following a collection of vignettes from schools around the country, readers weighed in. Here is a selection of their comments.'}